<a href="https://colab.research.google.com/github/NirobxHasan/COVID-19_Detection_from_Chest_X-Ray_Images/blob/main/CoronaHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

import helper
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/3bd7dea850e936d8cb44adda8200e4e2b5d627e3/intro-to-pytorch/helper.py
import importlib
importlib.reload(helper)

In [ ]:
#install depenencies
!pip install kaggle
#upload the credentials of the kaggle account, which is j JSON file.
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Give permissions to avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

In [ ]:
!unzip coronahack-chest-xraydataset.zip

In [ ]:
!ls

In [ ]:
import os
import pandas as pd
import time
import shutil

PATH_TRAIN = "/content/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/"
TOTAL_IMGS = len(os.listdir(PATH_TRAIN))

In [ ]:
import pandas as pd
data = pd.read_csv("/content/Chest_xray_Corona_Metadata.csv")
data

In [ ]:
# Printing unique labels
unique_labels = []
for i in  data['Label']:
    if i not in unique_labels:
        unique_labels.append(i)
        
print(unique_labels)


In [ ]:
import os
import pandas as pd
import time
import shutil

PATH_TRAIN = "/content/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train"
TOTAL_IMGS = len(os.listdir(PATH_TRAIN))
normal = 0
infected = 0



img = data["X_ray_image_name"]
label = data["Label"]
image_type = data["Dataset_type"]
all_dir = os.listdir(PATH_TRAIN)

os.mkdir("train")
os.mkdir('train/INFECTED')
os.mkdir("train/NORMAL")

wrong_info = 0  # Checking if the provided list maps the images correctly

# Moving the train images to designated folders

for i in range(len(image_type)):
    if image_type[i] == "TRAIN":
        if img[i] in all_dir: # Make sure that all images in Chest_xray_Corona_Metadata.csv is mapped
            if label[i] == "Normal":
                infected = infected + 1
                shutil.copy(
                    PATH_TRAIN + "/" + img[i], "train/NORMAL/" + img[i]
                )
                normal = normal + 1

            else:

                shutil.copy(
                    PATH_TRAIN + "/" + img[i], "train/INFECTED/" + img[i]
                )
                infected = infected + 1

        else:
            wrong_info = wrong_info + 1

print(
    "X-ray of Normal patients (TRAIN DATASET): " + str(normal),
    "X-ray of Infected patients (TRAIN DATASET): " + str(infected),
    end = "\n"
)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
categories = ["NORMAL", "INFECTED"]
number_of_imgs = [normal, infected]
ax.bar(0, number_of_imgs[0], color="g", width=0.1)
ax.bar(0.15, number_of_imgs[1], color="r", width=0.1)
ax.legend(labels=categories)
ax.set_ylabel("Number of images")
ax.set_xlabel("Categories")
plt.show()

In [ ]:
PATH_TEST = "/content/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test"
TOTAL_IMGS = len(os.listdir(PATH_TEST))
normal = 0
infected = 0

img = data["X_ray_image_name"]
label = data["Label"]
image_type = data["Dataset_type"]
all_dir = os.listdir(PATH_TEST)

os.mkdir("test")
os.mkdir('test/INFECTED')
os.mkdir("test/NORMAL")



wrong_info = 0

for i in range(len(image_type)):
    if image_type[i] == "TEST":
        if img[i] in all_dir:
            if label[i] == "Normal":
                infected = infected + 1
                shutil.copy(
                    PATH_TEST + "/" + img[i], "test/NORMAL/" + img[i]
                )
                normal = normal + 1

            else:

                shutil.copy(
                    PATH_TEST + "/" + img[i], "test/INFECTED/" + img[i]
                )
                infected = infected + 1

        else:
            wrong_info = wrong_info + 1

print(
    "X-ray of Normal patients (TEST DATASET): " + str(normal),
    "X-ray of Infected patients (TEST DATASET): " + str(infected),
    end = "\n"
)



In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
categories = ["NORMAL", "INFECTED"]
number_of_imgs = [normal, infected]
ax.bar(0, number_of_imgs[0], color="g", width=0.1)
ax.bar(0.15, number_of_imgs[1], color="r", width=0.1)
ax.legend(labels=categories)
ax.set_ylabel("Number of images")
ax.set_xlabel("Categories")
plt.show()

In [ ]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        n = 0
        for f in files:
            n = n+1
            if n>5:
                print('{}{}'.format(subindent, f),end = "  ...... \n")
                break
            print('{}{}'.format(subindent, f))


list_files("/content/Coronahack-Chest-XRay-Dataset")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import time
import torchvision
from PIL import ImageFile
import matplotlib.pyplot as plt
import cv2

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True # To prevent error during loading broken images

PATH_TRAIN = "train"
PATH_TEST  = "test"
EPOCHS = 10
BATCH_SIZE = 32
TOTAL_SIZE = len(os.listdir(PATH_TRAIN + "/NORMAL")) + len(
    os.listdir(PATH_TRAIN + "/INFECTED")
)
TOTAL_TEST_SIZE = len(os.listdir(PATH_TEST + "/NORMAL")) + len(
    os.listdir(PATH_TEST + "/INFECTED")
)
STEPS_PER_EPOCH = TOTAL_SIZE // BATCH_SIZE
STEPS_PER_TEST_EPOCH = TOTAL_TEST_SIZE // BATCH_SIZE
IMAGE_H, IMAGE_W = 224, 224

In [ ]:
transform = torchvision.transforms.Compose(
    [  # Applying Augmentation
        torchvision.transforms.Resize((IMAGE_H, IMAGE_W)),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.RandomVerticalFlip(p=0.5),
        torchvision.transforms.RandomRotation(30),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] # 0-1 to [-1,1] , formula (x-mean)/std
        ),
    ]
)  # Normalizing data

# Intitalizing the train data loader and applying the transformations

train_dataset = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=1, shuffle=True
)

# Intitalizing the test data loader

valid_dataset = torchvision.datasets.ImageFolder(
    root=PATH_TEST, transform=transform
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, num_workers=1, shuffle=True
)

In [ ]:
images, labels = next(iter(train_loader))
images.shape

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# #CNN network
# class Net(nn.Module):
#     def __init__(self,num_classes=2):
#         super(Net,self).__init__()
        
#         #Output size after convolution filter
#         #((w-f+2P)/s) +1
        
#         #Input shape= (32, 3, 224, 224)
        
#         self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,12,224,224)
#         # self.bn1=nn.BatchNorm2d(num_features=12)
#         #Shape= (256,12,224,224)
#         self.relu1=nn.ReLU()
#         #Shape= (256,12,224,224)
        
#         self.pool=nn.MaxPool2d(kernel_size=2)
#         #Reduce the image size be factor 2
#         #Shape= (256,12,112,112)
        
        
#         self.conv2=nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,20,112,112)
#         self.relu2=nn.ReLU()
#         #Shape= (256,20,112,112)
        
        
        
#         self.conv3=nn.Conv2d(in_channels=24,out_channels=64,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,32,112,112)
#         # self.bn3=nn.BatchNorm2d(num_features=64)
#         #Shape= (256,32,112,112)
#         self.relu3=nn.ReLU()
#         #Shape= (256,32,112,112)
        
        
#         self.fc1=nn.Linear(112 * 112 * 64, 500)
#         self.fc2=nn.Linear(500,out_features=num_classes)
        
#         self.dropout = nn.Dropout(0.25)
        
#         #Feed forwad function
        
#     def forward(self,input):
#         output=self.conv1(input)
#         # output=self.bn1(output)
#         output=self.relu1(output)
            
#         output=self.pool(output)
            
#         output=self.conv2(output)
#         output=self.relu2(output)
            
#         output=self.conv3(output)
#         # output=self.bn3(output)
#         output=self.relu3(output)
            
            
#             #Above output will be in matrix form, with shape (256,32,75,75)
            
#         output=output.view(-1,64*112*112)
            
#         output = self.dropout(output)
#         output=F.relu(self.fc1(output))
#         output = self.dropout(output)
#         output=self.fc2(output)
            
#         return output
# model = Net()
# print(model)

# # move tensors to GPU if CUDA is available
# if train_on_gpu:
#     model.cuda()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 5)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32*53*53, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 2)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 32 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

In [ ]:
# model=ConvNet(num_classes=2).to(device)

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [ ]:
# number of epochs to train the model
n_epochs = 20

valid_loss_min = np.Inf # track change in validation loss
train_losses, test_losses = [], []
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
  
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    accuracy = 0 
    model.eval()

    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

        ps = torch.exp(output)
        top_p,top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape).cuda()
        accuracy += torch.mean(equals.type(torch.FloatTensor))
     
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)

    train_losses.append(train_loss)
    test_losses.append(valid_loss)


    # # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t Test Accuracy: {:.3f}'.format(
        epoch, train_loss, valid_loss,accuracy/len(valid_loader)))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'CoronaHack_2layer.pth')
        valid_loss_min = valid_loss

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

Resnet18 implementation

In [ ]:
model_ft = torchvision.models.resnet18(False)  # Initializing resnet18
model_ft.load_state_dict(torch.load("/content/resnet18-5c106cde.pth"))
num_ftrs = model_ft.fc.in_features # Getting last layer's output features
model_ft.fc = nn.Linear(num_ftrs, 2) # Modifying the last layer accordng to our need

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.to(device)  # Sending model to device
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model_ft.parameters(), lr=0.0007
)  # lr should be kept low so that the pre-trained weights don't change easily

In [ ]:
def get_test():
    test_loss = []
    correct = 0
    incorrect = 0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for batch_idx, (data, target) in enumerate(valid_loader):
        if batch_idx == STEPS_PER_TEST_EPOCH:
            break

       

        with torch.no_grad():

            data = data.to(device)
            target = target.long().to(device)
            output = model_ft(data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

            

            for i in range(BATCH_SIZE):
                a = []
                for j in output[i]:
                    a.append(float(j.detach()))

                pred = a.index(max(a))

                if pred == int(target[i]):
                    correct = correct + 1

                else:
                    incorrect = incorrect + 1

        test_loss.append(float(loss.detach()))
    print("CORRECT: " + str(correct), "INCORRECT: " + str(incorrect),"TEST ACCURACY: "+str(correct/(correct+incorrect)))
    return (
            correct/(incorrect+correct),
            sum(test_loss)/len(test_loss),
    )

In [ ]:
acc_ , loss_ = get_test()
print("ACCURACY AND LOSS BEFORE TUNING")
print("ACCURACY : "+str(acc_),"LOSS : "+str(loss_))

In [ ]:
avg_test_loss_history = []
avg_test_accuracy_history = []
avg_train_loss_history = []
avg_train_accuracy_history = []

loss_history = []
accuracy_history = []

new_best = 0

for i in range(EPOCHS):

    start = time.time()
    print(
        "-----------------------EPOCH "
        + str(i)
        + " -----------------------------------"
    )
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx == STEPS_PER_EPOCH:
            break
        optimizer.zero_grad()  # Resetting gradients after each optimizations
        # Sending input , target to device
        data = data.to(device) 
        target = target.to(device)
        output = model_ft(data)
        loss = criterion(output, target.reshape((BATCH_SIZE,)).long())
        loss_history.append(loss.detach())
       
        loss.backward()
        optimizer.step()  # Optimizing the model

        # Checking train accuracy

        correct = 0
        incorrect = 0
        for p in range(BATCH_SIZE):
            a = []
            for j in output[p]:
                a.append(float(j.detach()))

            pred = a.index(max(a))

            if pred == int(target[p]):
                correct = correct + 1

            else:

                incorrect = incorrect + 1

        print(
            "\r EPOCH "
            + str(i)
            + " MINIBATCH: "
            + str(batch_idx)
            + "/"
            + str(STEPS_PER_EPOCH)
            + " LOSS: "
            + str(loss_history[-1]),
            end = ""
            
        )
        
        accuracy_history.append(correct/(correct+incorrect))

    end = time.time()
    print(
        " \n EPOCH "
        + str(i)
        + " LOSS "
        + str(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
        + " ETA: "
        + str(end - start)
        + " \n MAX LOSS: "
        + str(max(loss_history[-STEPS_PER_EPOCH:]))
        + " MIN LOSS: "
        + str(min(loss_history[-STEPS_PER_EPOCH:]))
        + " TRAIN ACCURACY: "
        + str(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    )
    
    avg_train_loss_history.append(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    avg_train_accuracy_history.append(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    
    test_acc , test_loss  = get_test()
    
    avg_test_accuracy_history.append(test_acc)
    avg_train_loss_history.append(test_loss)
    
    if test_acc>new_best: 
        new_best = test_acc
        torch.save(model_ft.state_dict(), "model_resnet18.pth") # Saving our best model

In [ ]:
model_ft.load_state_dict(torch.load('model_resnet18.pth'))

In [ ]:
print("ACCURACY : ",new_best)

In [ ]:
plt.plot(avg_train_loss_history,label = "Train")
plt.plot(avg_test_loss_history , label = "Test")
plt.title('LOSS PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("LOSS")

In [ ]:
plt.plot(avg_train_accuracy_history , label = "Train")
plt.plot(avg_test_accuracy_history , label = "Test")
plt.title('ACCURACY PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("ACCURACY")